[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harpreetsahota204/medgemma_1_5/blob/main/using_medgemma_on_slake.ipynb)


# Using MedGemma as Remotely Sourced Zoo Model

In [ ]:
!pip install fiftyone accelerate

In [ ]:
import fiftyone as fo

from fiftyone.utils.huggingface import load_from_hub

dataset = load_from_hub(
    "Voxel51/SLAKE",
    name="SLAKE",
    overwrite=True,
    max_samples=50
    )

# Setup Zoo Model

In [ ]:
import fiftyone.zoo as foz

foz.register_zoo_model_source("https://github.com/harpreetsahota204/medgemma_1_5", overwrite=True)

foz.download_zoo_model(
    "https://github.com/harpreetsahota204/medgemma_1_5",
    model_name="google/medgemma-1.5-4b-it", 
)

model = foz.load_zoo_model(
    "google/medgemma-1.5-4b-it",
    # install_requirements=True #run this to install requirements if they're not already
    )

# Use MedGemma for Classification

You can use this model for a zero-shot classification task as follows, which will add a [FiftyOne Classificaton](https://docs.voxel51.com/api/fiftyone.core.labels.html#fiftyone.core.labels.Classification) to your dataset:

In [ ]:
body_system_labels = dataset.distinct("modality.label")

model.operation = "classify"

model.prompt = "As a medical expert your task is to classify this image into exactly one of the following types: " + ", ".join(body_system_labels)

dataset.apply_model(model, label_field="pred_modality")

In [ ]:
dataset.first()

Here's the ground truth:

In [ ]:
dataset.first()['modality']

And the model prediction:

In [ ]:
dataset.first()['pred_modality.classifications']

# Using MedGemma for VQA

You can use the model for visual question answering as shown below. This example will use the same prompt on each [Sample](https://docs.voxel51.com/api/fiftyone.core.sample.html#module-fiftyone.core.sample) in the Dataset. Note the default system prompt:

In [ ]:
print(model.system_prompt)

You can customize the system prompt as follows to guide the model's response. Note that we are using an existing field on the sample by passing `prompt_field="question"` into the [`apply_model`](https://docs.voxel51.com/api/fiftyone.core.dataset.html) method of the [Dataset](https://docs.voxel51.com/api/fiftyone.core.dataset.html).

Note that if you want to parse the model output as a [FiftyOne Classification](https://docs.voxel51.com/api/fiftyone.core.labels.html#fiftyone.core.labels.Classification) then you need to very specifically prompt the model to output in a way that this integration expects, that is:


```json
{
    "classifications": [
        {
            "label": "your answer to the question",
            ...,
        }
    ]
}
```

Notice below:

In [ ]:
model.operation="classify"

model.system_prompt = """You have expert-level medical knowledge in radiology, histopathology, ophthalmology, and dermatology.

You will be asked a question and are required to provide your answer. Your answer must be in the following format:

```json
{
    "classifications": [
        {
            "label": "your answer to the question",
            ...,
        }
    ]
}
```

Always return your response as valid JSON wrapped in ```json blocks and respond only with one answer.
"""

dataset.apply_model(
    model, 
    label_field="pred_answer_6", 
    prompt_field="question_6"
    )

In [ ]:
dataset.first()['question_6']

In [ ]:
dataset.first()['answer_6']

In [ ]:
dataset.first()['pred_answer_6.classifications']

For open-ended generation, you can use `vqa` mode. Note that with both modes you can use a single question on each sample as seen below:

In [ ]:
model.system_prompt = None # we need to clear the custom system prompt  

model.operation="vqa"

model.prompt = "Describe any anomolies in this the image that you observe."

dataset.apply_model(model, label_field="open_response")

In [ ]:
print(dataset.first()["open_response"])

Or use you can use a Sample field:

In [ ]:
model.operation="vqa"

dataset.apply_model(
    model, 
    label_field="answer_5_vqa", 
    prompt_field="question_5"
    )

In [ ]:
dataset.first()["question_5"]

In [ ]:
dataset.first()["answer_5"]

In [ ]:
dataset.first()["answer_5_vqa"]

# Evaluating the model

You can use FiftyOne's [Evaluation API](https://docs.voxel51.com/user_guide/evaluation.html) to evaluate model performance via the SDK. 

For example:

In [ ]:
results = dataset.evaluate_classifications(
    "pred_modality",
    gt_field="modality",
    eval_key="eval_simple",
)

You can then review as follows:

In [ ]:
results.print_report()

In [ ]:
plot = results.plot_confusion_matrix()
plot.show()

You can, of course, do all of this in the App as shown here: